In [1]:
#Imports
import logging
import os, random, time, sys
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers
import ipdb
import keras
import pandas as pd

sys.path.append('../../../')
import feedforward_robust as ffr

sys.path.append('../../../utils/')
from utils.mnist_corruption import *
from utils.utils_models import *
from utils.utils_analysis import *
from utils.utils_feedforward import *

%load_ext autoreload
%autoreload 2

#Read the counter
ctr_file = "../../counter.txt"
f = open(ctr_file, 'r')
counter = f.readline()
f.close()

counter = 1 + int(counter)
f = open(ctr_file,'w')
f.write('{}'.format(counter))
f.close()
logfile = "../../logs/results_" + str(counter) + ".log"

logger = logging.getLogger("robustness")
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler(logfile)
fh.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)
logger.addHandler(logging.StreamHandler())

Using TensorFlow backend.
W0611 18:10:02.297460 4393846208 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/cleverhans/utils_tf.py:341: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.



# Get Fashion MNIST Data

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train/255
x_test = x_test/255

In [4]:
y_train.shape

(60000,)

In [5]:
#One hot the labels
num_classes = 10
y_train_oh = keras.utils.to_categorical(y_train, num_classes)                                                                                         
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
y_train_oh.shape

(60000, 10)

In [7]:
max_val = 0.2
y_train_oh = y_train_oh * max_val
remainder = (1 - max_val)/9.0

In [8]:
remainder

0.08888888888888889

In [9]:
to_add = np.ones(y_train_oh.shape)
for i in range(len(y_train_oh)):
    correct_index = y_train[i]
    to_add[i, correct_index] = 0
to_add = to_add * remainder
y_train_oh = y_train_oh + to_add

In [10]:
y_train = y_train_oh

In [11]:
x_train_master, y_train_master = x_train[:30000], y_train[:30000]
x_train_slave, y_train_slave = x_train[30000:], y_train[30000:]

In [12]:
#Flatten everything
x_train_master_flat, input_shape = flatten_mnist(x_train_master) 
x_train_slave_flat, _ = flatten_mnist(x_train_slave)
x_test_flat, _  = flatten_mnist(x_test)

# Train Master

In [13]:
#Configurations
eps_train = 0.1                                                                                                                            
eps_test = 0.1                                                                                                                             
tensorboard_dir = "../tb/"                                                                                                                
weights_dir = "../weights/"                                                                                                               
load_weights = False                                                                                                              
load_counter = 234                                                                                                            
sigma = tf.nn.relu                                                                                                                         
epochs, reg, lr = 100, 0.00, 1e-3    
#epochs, reg, lr = 30, 0.00, 15e-4                                                                                                          
pgd_eta, pgd_num_iter = 1e-2, 50

In [14]:
#Setup - Dataset stuff
tf.reset_default_graph()
sess = tf.Session()
hidden_sizes = [64, 64, 32]
dataset = ((x_train_master_flat, y_train_master), (x_test_flat, y_test))

scope_name = "teacher_student_fashion"
if not load_weights:
    with tf.variable_scope(scope_name, reuse = False) as scope:

        logdir = tensorboard_dir + str(counter)

        #Create model
        writer = tf.summary.FileWriter(logdir)
        model = ffr.RobustMLP(input_shape, hidden_sizes, num_classes, writer = writer, scope = scope_name, logger = logger, sigma = sigma)
        sess.run(tf.global_variables_initializer())
        print("Created model successfully. Now going to train")
    
        #Train model
        model.fit(sess, x_train_master_flat, y_train_master, training_epochs = epochs, reg_op = reg , lr = lr)
        
        """
        #Save weights
        weights = tf.trainable_variables()
        #weights = model.get_weights()[0] + model.get_weights()[1]
        saver = tf.train.Saver(weights)
        weights_path = saver.save(sess, weights_dir + "model_" + str(counter) + ".ckpt")
        print("Saved model at %s"%weights_path)
        """
        
        weight_norms = model.get_weight_norms(sess)
        print("----- Weight norms ----")
        print(weight_norms)
        
        #Test model - regular, fgsm adv, pgd adv
        loss_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
        print("----Regular test accuracy and loss ----")
        print((loss_reg, acc_reg))
        
        loss_fgsm, acc_fgsm = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = False)
        print("----FGSM test accuracy and loss ----")
        print((loss_fgsm, acc_fgsm))
        
        loss_pgd, acc_pgd = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = True, eta=pgd_eta, num_iter = pgd_num_iter)
        print("----PGD test accuracy and loss ----")
        print((loss_pgd , acc_pgd))

Initialized instance variables of the robust model class
I0611 18:10:03.840367 4393846208 feedforward_robust.py:32] Initialized instance variables of the robust model class
W0611 18:10:03.844049 4393846208 deprecation_wrapper.py:119] From ../../../feedforward_robust.py:36: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

Created placeholders for x and y
I0611 18:10:03.856899 4393846208 feedforward_robust.py:40] Created placeholders for x and y
W0611 18:10:03.858345 4393846208 deprecation_wrapper.py:119] From ../../../utils/utils_feedforward.py:34: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0611 18:10:03.859033 4393846208 deprecation_wrapper.py:119] From ../../../utils/utils_feedforward.py:34: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0611 18:10:03.860620 4393846208 deprecation_wrapper.py:119] From ../../../utils/utils_feedforward.py:36: The name tf.get_variable

Created model successfully. Now going to train


W0611 18:10:04.570297 4393846208 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py:193: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
Epoch: 0001    cost: 2.274303485 
I0611 18:10:05.999381 4393846208 feedforward_robust.py:716] Epoch: 0001    cost: 2.274303485 
Accuracy on batch: 0.875000
I0611 18:10:06.001448 4393846208 feedforward_robust.py:717] Accuracy on batch: 0.875000
Epoch: 0002    cost: 2.268948558 
I0611 18:10:07.102023 4393846208 feedforward_robust.py:716] Epoch: 0002    cost: 2.268948558 
Accuracy on batch: 0.843750
I0611 18:10:07.103872 4393846208 feedforward_robust.py:717] Accuracy on batch: 0.843750
Epoch: 0003    cost: 2.267758763 
I0611 18:10:08.224104 4393846208 feedforward_robust.py:716] Epoch: 0003    cost: 2.267758763 
Accuracy on batch: 0.906250
I0611 18:10:08.225656 4393846208 fe

Accuracy on batch: 0.906250
I0611 18:10:44.712272 4393846208 feedforward_robust.py:717] Accuracy on batch: 0.906250
Epoch: 0034    cost: 2.262851099 
I0611 18:10:46.218579 4393846208 feedforward_robust.py:716] Epoch: 0034    cost: 2.262851099 
Accuracy on batch: 0.875000
I0611 18:10:46.220253 4393846208 feedforward_robust.py:717] Accuracy on batch: 0.875000
Epoch: 0035    cost: 2.262796008 
I0611 18:10:47.488276 4393846208 feedforward_robust.py:716] Epoch: 0035    cost: 2.262796008 
Accuracy on batch: 0.906250
I0611 18:10:47.491799 4393846208 feedforward_robust.py:717] Accuracy on batch: 0.906250
Epoch: 0036    cost: 2.262823763 
I0611 18:10:48.653672 4393846208 feedforward_robust.py:716] Epoch: 0036    cost: 2.262823763 
Accuracy on batch: 0.843750
I0611 18:10:48.657068 4393846208 feedforward_robust.py:717] Accuracy on batch: 0.843750
Epoch: 0037    cost: 2.262720877 
I0611 18:10:49.947056 4393846208 feedforward_robust.py:716] Epoch: 0037    cost: 2.262720877 
Accuracy on batch: 0.906

I0611 18:11:26.913780 4393846208 feedforward_robust.py:716] Epoch: 0067    cost: 2.261756406 
Accuracy on batch: 1.000000
I0611 18:11:26.915063 4393846208 feedforward_robust.py:717] Accuracy on batch: 1.000000
Epoch: 0068    cost: 2.261788296 
I0611 18:11:28.147523 4393846208 feedforward_robust.py:716] Epoch: 0068    cost: 2.261788296 
Accuracy on batch: 1.000000
I0611 18:11:28.149203 4393846208 feedforward_robust.py:717] Accuracy on batch: 1.000000
Epoch: 0069    cost: 2.261773589 
I0611 18:11:29.342586 4393846208 feedforward_robust.py:716] Epoch: 0069    cost: 2.261773589 
Accuracy on batch: 0.968750
I0611 18:11:29.343822 4393846208 feedforward_robust.py:717] Accuracy on batch: 0.968750
Epoch: 0070    cost: 2.261727954 
I0611 18:11:30.522279 4393846208 feedforward_robust.py:716] Epoch: 0070    cost: 2.261727954 
Accuracy on batch: 1.000000
I0611 18:11:30.523824 4393846208 feedforward_robust.py:717] Accuracy on batch: 1.000000
Epoch: 0071    cost: 2.261739371 
I0611 18:11:31.648666 43

...

----- Weight norms ----
[81.77506, 21.89681, 19.51775, 2.9443429]
----Regular test accuracy and loss ----
(1.7286503, 0.8634)


Model is being evaluated on FGSM data
I0611 18:12:09.977263 4393846208 feedforward_robust.py:649] Model is being evaluated on FGSM data
Model is being evaluated on PGD points generated using 0.010000 learning rate and 50 iterations
I0611 18:12:10.006647 4393846208 feedforward_robust.py:651] Model is being evaluated on PGD points generated using 0.010000 learning rate and 50 iterations


----FGSM test accuracy and loss ----
(2.1431105, 0.3622)
iteration: 0
loss 1.850557
iteration: 1
loss 1.930215
iteration: 2
loss 1.998748
iteration: 3
loss 2.051919
iteration: 4
loss 2.095126
iteration: 5
loss 2.130426
iteration: 6
loss 2.161141
iteration: 7
loss 2.185529
iteration: 8
loss 2.208078
iteration: 9
loss 2.228520
iteration: 10
loss 2.246605
iteration: 11
loss 2.262784
iteration: 12
loss 2.278572
iteration: 13
loss 2.293634
iteration: 14
loss 2.307968
iteration: 15
loss 2.322614
iteration: 16
loss 2.331178
iteration: 17
loss 2.345902
iteration: 18
loss 2.354377
iteration: 19
loss 2.364953
iteration: 20
loss 2.373450
iteration: 21
loss 2.382411
iteration: 22
loss 2.391316
iteration: 23
loss 2.397739
iteration: 24
loss 2.405911
iteration: 25
loss 2.411819
iteration: 26
loss 2.418280
iteration: 27
loss 2.424141
iteration: 28
loss 2.430579
iteration: 29
loss 2.436071
iteration: 30
loss 2.442919
iteration: 31
loss 2.446775
iteration: 32
loss 2.452252
iteration: 33
loss 2.455638
i

This is to confirm that attack does not violate constraints
I0611 18:12:18.463122 4393846208 feedforward_robust.py:496] This is to confirm that attack does not violate constraints
Should be no more than eps
I0611 18:12:18.466264 4393846208 feedforward_robust.py:497] Should be no more than eps
0.10000005352730845
I0611 18:12:18.565474 4393846208 feedforward_robust.py:498] 0.10000005352730845


----PGD test accuracy and loss ----
(2.5120447, 0.2442)


In [15]:
a

NameError: name 'a' is not defined

## Generate data for slave

In [ ]:
with tf.variable_scope(scope_name, reuse = False) as scope:
        loss_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
        print("----Regular test accuracy and loss ----")
        print((loss_reg, acc_reg))
        
        z_train_slave = model.get_prediction(sess, x_train_slave_flat)
        z_test_slave = model.get_prediction(sess, x_test_flat)

In [ ]:
z_train_slave[0]

# Train slave

In [ ]:
#Setup - Dataset stuff
def slave_training():
    epochs = 30
    lr = 15e-4
    tf.reset_default_graph()
    sess = tf.Session()
    hidden_sizes = [64, 64, 32]
    dataset = ((x_train_slave_flat, z_train_slave), (x_test_flat, y_test))

    scope_name = "teacher_student_fashion"
    if not load_weights:
        with tf.variable_scope(scope_name, reuse = tf.AUTO_REUSE) as scope:

            logdir = tensorboard_dir + str(counter)

            #Create model
            writer = tf.summary.FileWriter(logdir)
            model = ffr.RobustMLP(input_shape, hidden_sizes, num_classes, writer = writer, scope = scope_name, logger = logger, sigma = sigma, classification = False)
            sess.run(tf.global_variables_initializer())
            print("Created model successfully. Now going to train")

            #Train model
            model.fit(sess, x_train_slave_flat, z_train_slave, training_epochs = epochs, reg_op = reg , lr = lr)

            """
            #Save weights
            weights = tf.trainable_variables()
            #weights = model.get_weights()[0] + model.get_weights()[1]
            saver = tf.train.Saver(weights)
            weights_path = saver.save(sess, weights_dir + "model_" + str(counter) + ".ckpt")
            print("Saved model at %s"%weights_path)
            """
            loss_real_train, acc_train = model.evaluate(sess, x_train_slave_flat, z_train_slave)

            #Test model - regular, fgsm adv, pgd adv
            loss_real_reg, acc_real_reg = model.evaluate(sess, x_test_flat, z_test_slave)
            print("----Regular test loss and accuracy ----")
            print((loss_real_reg, acc_real_reg))
            
            loss_class_reg, acc_reg = model.evaluate(sess, x_test_flat, y_test)
            print("----Real test loss and accuracy comparing to teacher ----")
            print((loss_class_reg, acc_reg))

            loss_fgsm, acc_fgsm = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = False)
            print("----FGSM test loss and accuracy ----")
            print((loss_fgsm, acc_fgsm))

            loss_pgd, acc_pgd = model.adv_evaluate(sess, x_test_flat, y_test, eps_test, pgd = True, eta=pgd_eta, num_iter = pgd_num_iter)
            print("----PGD test loss and accuracy ----")
            print((loss_pgd , acc_pgd))
            
            slave_train_confidences = model.get_prediction(sess, x_train_slave_flat)
            slave_test_confidences = model.get_prediction(sess, x_test_flat)
            
            return loss_real_train, acc_train, loss_real_reg, acc_real_reg, acc_reg, slave_train_confidences, slave_test_confidences
        
        

In [ ]:
tup = slave_training()

In [ ]:
list(tup)

In [ ]:
df = pd.DataFrame()
df['mse on z_train'] = []
df['acc on z_train'] = []
df['mse on z_test'] = []
df['acc on z_test'] = []
df['acc on y_test'] = []

In [ ]:
train_confidences = []
test_confidences = []
for i in range(3):
    loss_real_train, acc_train, loss_real_reg, acc_real_reg, acc_reg, slave_train_confidences, slave_test_confidences = slave_training()
    df.loc[i] = [loss_real_train, acc_train, loss_real_reg, acc_real_reg, acc_reg]
    train_confidences.append(slave_train_confidences)
    test_confidences.append(slave_test_confidences)

In [ ]:
df

In [ ]:
#df.to_excel("ts_fashion_results.xlsx")  